# **Task_04 : Image-to-Image Translation with cGAN**

*Implement an image-to-image translation model using a conditional generative adversarial network (cGAN) called pix2pix.*

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
pip install torch torchvision matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import os


In [3]:
class UNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, act='relu', use_dropout=False):
        super().__init__()
        self.down = down
        self.act = nn.ReLU() if act == 'relu' else nn.LeakyReLU(0.2)
        self.conv = nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False) if down \
                    else nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False)
        self.norm = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout(0.5) if use_dropout else nn.Identity()

    def forward(self, x):
        x = self.conv(x)
        x = self.norm(x)
        x = self.dropout(x)
        return self.act(x)

class GeneratorUNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super().__init__()

        self.down1 = UNetBlock(in_channels, 64, act='lrelu', use_dropout=False)
        self.down2 = UNetBlock(64, 128, act='lrelu')
        self.down3 = UNetBlock(128, 256, act='lrelu')
        self.down4 = UNetBlock(256, 512, act='lrelu')
        self.down5 = UNetBlock(512, 512, act='lrelu')
        self.down6 = UNetBlock(512, 512, act='lrelu')
        self.down7 = UNetBlock(512, 512, act='lrelu')
        self.bottleneck = UNetBlock(512, 512, act='relu')

        self.up1 = UNetBlock(512, 512, down=False, use_dropout=True)
        self.up2 = UNetBlock(1024, 512, down=False, use_dropout=True)
        self.up3 = UNetBlock(1024, 512, down=False, use_dropout=True)
        self.up4 = UNetBlock(1024, 512, down=False)
        self.up5 = UNetBlock(1024, 256, down=False)
        self.up6 = UNetBlock(512, 128, down=False)
        self.up7 = UNetBlock(256, 64, down=False)

        self.final = nn.Sequential(
            nn.ConvTranspose2d(128, out_channels, 4, 2, 1),
            nn.Tanh()
        )

    def forward(self, x):
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        d7 = self.down7(d6)
        bn = self.bottleneck(d7)

        u1 = self.up1(bn)
        u2 = self.up2(torch.cat([u1, d7], 1))
        u3 = self.up3(torch.cat([u2, d6], 1))
        u4 = self.up4(torch.cat([u3, d5], 1))
        u5 = self.up5(torch.cat([u4, d4], 1))
        u6 = self.up6(torch.cat([u5, d3], 1))
        u7 = self.up7(torch.cat([u6, d2], 1))

        return self.final(torch.cat([u7, d1], 1))


In [4]:
class Discriminator(nn.Module):
    def __init__(self, in_channels=3):
        super().__init__()
        def block(in_c, out_c, norm=True):
            layers = [nn.Conv2d(in_c, out_c, 4, 2, 1)]
            if norm:
                layers.append(nn.BatchNorm2d(out_c))
            layers.append(nn.LeakyReLU(0.2))
            return layers

        self.model = nn.Sequential(
            *block(in_channels * 2, 64, norm=False),
            *block(64, 128),
            *block(128, 256),
            *block(256, 512),
            nn.Conv2d(512, 1, 4, 1, 1)  # Patch output
        )

    def forward(self, x, y):
        # x: input image, y: target or generated image
        return self.model(torch.cat([x, y], dim=1))


In [5]:
criterion_GAN = nn.MSELoss()
criterion_L1 = nn.L1Loss()


In [6]:
def train(dataloader, generator, discriminator, g_optimizer, d_optimizer, device):
    generator.train()
    discriminator.train()

    for epoch in range(100):
        for i, (input_image, target_image) in enumerate(dataloader):
            input_image = input_image.to(device)
            target_image = target_image.to(device)

            # ---------------------
            #  Train Discriminator
            # ---------------------
            fake_image = generator(input_image)
            real_pred = discriminator(input_image, target_image)
            fake_pred = discriminator(input_image, fake_image.detach())

            real_loss = criterion_GAN(real_pred, torch.ones_like(real_pred))
            fake_loss = criterion_GAN(fake_pred, torch.zeros_like(fake_pred))
            d_loss = (real_loss + fake_loss) * 0.5

            d_optimizer.zero_grad()
            d_loss.backward()
            d_optimizer.step()

            # -----------------
            #  Train Generator
            # -----------------
            fake_pred = discriminator(input_image, fake_image)
            gan_loss = criterion_GAN(fake_pred, torch.ones_like(fake_pred))
            l1 = criterion_L1(fake_image, target_image) * 100

            g_loss = gan_loss + l1

            g_optimizer.zero_grad()
            g_loss.backward()
            g_optimizer.step()

        print(f"Epoch [{epoch}] | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}")
        save_image(fake_image[:4], f"outputs/fake_{epoch}.png", nrow=2, normalize=True)


In [8]:
from PIL import Image
from torch.utils.data import Dataset
import os

class PairedImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.input_dir = os.path.join(root_dir, "input")
        self.target_dir = os.path.join(root_dir, "target")
        self.filenames = sorted(os.listdir(self.input_dir))
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        input_path = os.path.join(self.input_dir, self.filenames[idx])
        target_path = os.path.join(self.target_dir, self.filenames[idx])

        input_image = Image.open(input_path).convert("RGB")
        target_image = Image.open(target_path).convert("RGB")

        if self.transform:
            input_image = self.transform(input_image)
            target_image = self.transform(target_image)

        return input_image, target_image


In [11]:
from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = PairedImageDataset("/content/drive/MyDrive/pix2pix_sample_paired_dataset", transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
G = GeneratorUNet().to(device)
D = Discriminator().to(device)

g_optimizer = optim.Adam(G.parameters(), lr=2e-4, betas=(0.5, 0.999))
d_optimizer = optim.Adam(D.parameters(), lr=2e-4, betas=(0.5, 0.999))

os.makedirs("outputs", exist_ok=True)
train(dataloader, G, D, g_optimizer, d_optimizer, device)


Epoch [0] | D Loss: 0.3491 | G Loss: 47.8418
Epoch [1] | D Loss: 0.1102 | G Loss: 36.4447
Epoch [2] | D Loss: 0.1326 | G Loss: 32.4275
Epoch [3] | D Loss: 0.2928 | G Loss: 22.4157
Epoch [4] | D Loss: 0.1273 | G Loss: 18.9906
Epoch [5] | D Loss: 0.0692 | G Loss: 18.6607
Epoch [6] | D Loss: 0.0633 | G Loss: 16.4297
Epoch [7] | D Loss: 0.0845 | G Loss: 16.1534
Epoch [8] | D Loss: 0.0771 | G Loss: 13.9206
Epoch [9] | D Loss: 0.1926 | G Loss: 14.6335
Epoch [10] | D Loss: 0.0414 | G Loss: 13.0245
Epoch [11] | D Loss: 0.0472 | G Loss: 12.0464
Epoch [12] | D Loss: 0.0720 | G Loss: 11.1387
Epoch [13] | D Loss: 0.0496 | G Loss: 11.6159
Epoch [14] | D Loss: 0.3072 | G Loss: 10.0608
Epoch [15] | D Loss: 0.1686 | G Loss: 10.6748
Epoch [16] | D Loss: 0.0225 | G Loss: 10.4104
Epoch [17] | D Loss: 0.0572 | G Loss: 10.2415
Epoch [18] | D Loss: 0.0776 | G Loss: 10.1839
Epoch [19] | D Loss: 0.1796 | G Loss: 9.9946
Epoch [20] | D Loss: 0.1805 | G Loss: 9.0807
Epoch [21] | D Loss: 0.0944 | G Loss: 9.2263
E